In [2]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: multitasking>=0.0.7 in c:\users\ruben\appdata\local\programs\python\python310\lib\site-packages (from yfinance) (0.0.10)


You should consider upgrading via the 'c:\Users\Ruben\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [11]:
import pandas as pd
import numpy as np
import yfinance as yf


In [18]:
stock_symbols_ftse_mib = ["A2A.MI", "AMP.MI", "ATL.MI", "AZM.MI", "BAMI.MI", "BGN.MI", "BMED.MI", "BPE.MI", "BZU.MI", "CNHI.MI", "CPR.MI", "DIA.MI", "ENEL.MI", "ENI.MI", "EXO.MI", "FBK.MI", "G.MI", "HER.MI",
                          "IG.MI", "INW.MI", "IP.MI", "ISP.MI", "LDO.MI", "MB.MI", "MONC.MI", "NEXI.MI", "PIRC.MI", "PRY.MI", "PST.MI", "RACE.MI", "REC.MI", "SMP.MI", "SRG.MI", "STLA.MI", "STM.MI", "TEN.MI", "TIT.MI", "TRN.MI", "UCG.MI", "UNI.MI", "FTSEMIB.MI"]


In [19]:
for stock in stock_symbols_ftse_mib:
  df = yf.download(stock, start="2017-06-08", end="2022-06-08")
  df.to_csv("./Datasets/"+stock+".csv")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Calculate log returns for each stock

In [16]:
test = pd.read_csv("./Datasets/ATL.MI.csv")

In [17]:
test.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,log_ret
0,0,2017-06-08,25.860001,26.090000,25.620001,25.910000,23.019947,3295679,NaN
1,1,2017-06-09,25.930000,26.070000,25.790001,26.000000,23.099909,2182808,0.003468
2,2,2017-06-12,26.020000,26.020000,25.650000,25.650000,22.788946,1748006,-0.013553
3,3,2017-06-13,25.750000,25.959999,25.629999,25.959999,23.064371,2182772,0.012013
4,4,2017-06-14,25.900000,26.290001,25.900000,26.180000,23.259832,2761136,0.008439


In [20]:
for stock in stock_symbols_ftse_mib:
  df = pd.read_csv("./Datasets/"+stock+".csv")
  df["log_ret"] = np.log(df["Close"]) - np.log(df["Close"].shift(1))
  df.to_csv("./Datasets/"+stock+".csv")

In [21]:
import statsmodels.api as sm
from statsmodels import regression

In [28]:
df = pd.read_csv("./Datasets/"+"A2A.MI"+".csv")
df_ftsemib = pd.read_csv("./Datasets/"+"FTSEMIB.MI"+".csv")
df = df.dropna()
df_ftsemib = df_ftsemib.dropna()
rolling_number = 0
df_ftsemib = df_ftsemib[rolling_number: rolling_number+180]
df = df[rolling_number: rolling_number+180]


In [29]:
df.shape

(180, 9)

In [30]:
df_ftsemib.shape

(180, 9)

In [36]:
y = df["log_ret"]
X = df_ftsemib["log_ret"]
X = sm.add_constant(X)
model = regression.linear_model.OLS(y, X).fit()
print(model.summary())
print("Alpha: %s - Beta: %s" % (model.params[0], model.params[1]))


                            OLS Regression Results                            
Dep. Variable:                log_ret   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     73.97
Date:                Wed, 08 Jun 2022   Prob (F-statistic):           4.01e-15
Time:                        17:50:09   Log-Likelihood:                 572.49
No. Observations:                 180   AIC:                            -1141.
Df Residuals:                     178   BIC:                            -1135.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0007      0.001     -0.944      0.3

In [37]:
from tqdm import tqdm
import os


In [47]:
df_ftsemib = pd.read_csv("./Datasets/"+"FTSEMIB.MI"+".csv")
df_ftsemib = df_ftsemib.dropna()
for stock in tqdm(stock_symbols_ftse_mib):
  df = pd.read_csv("./Datasets/"+stock+".csv")
  df = df.dropna()
  for time_index in tqdm(range(0, int(len(df_ftsemib)/180))):
    time_window_stock = df[time_index: time_index+180]
    time_window_ftsemib = df_ftsemib[time_index: time_index+180]
   
    X = time_window_ftsemib["log_ret"]
    y = time_window_stock["log_ret"]
    X = sm.add_constant(X)
    model = regression.linear_model.OLS(y, X).fit()
    if not os.path.exists("./Models/"+stock):
      os.makedirs("./Models/"+stock)
    model.save("./Models/"+stock+"/"+str(time_index)+".pickle")
    
    

  2%|▏         | 1/41 [00:00<00:05,  7.09it/s]

   Unnamed: 0        Date   Open   High    Low  Close  Adj Close    Volume  \
1           1  2017-06-09  1.495  1.508  1.486  1.492   1.161318   5511144   
2           2  2017-06-12  1.495  1.500  1.485  1.490   1.159761   6551209   
3           3  2017-06-13  1.492  1.505  1.480  1.482   1.153534  10918022   
4           4  2017-06-14  1.483  1.499  1.480  1.489   1.158983   4822536   
5           5  2017-06-15  1.490  1.505  1.471  1.495   1.163653  12471585   

    log_ret  
1 -0.005348  
2 -0.001341  
3 -0.005384  
4  0.004712  
5  0.004021  
   Unnamed: 0        Date     Open     High      Low    Close  Adj Close  \
1           1  2017-06-09  21045.0  21271.0  20948.0  21122.0    21122.0   
2           2  2017-06-12  21077.0  21157.0  20910.0  20910.0    20910.0   
3           3  2017-06-13  20989.0  21169.0  20949.0  21089.0    21089.0   
4           4  2017-06-14  21095.0  21198.0  20961.0  20961.0    20961.0   
5           5  2017-06-15  20929.0  20937.0  20732.0  20848.0    20

   Unnamed: 0        Date   Open   High    Low  Close  Adj Close   Volume  \
1           1  2017-06-09  12.91  13.13  12.91  13.00  12.544929   680971   
2           2  2017-06-12  13.00  13.00  12.49  12.65  12.207181   680831   
3           3  2017-06-13  12.74  12.90  12.65  12.90  12.448428   467243   
4           4  2017-06-14  12.06  12.24  11.95  12.20  11.772934  7618194   
5           5  2017-06-15  12.20  12.23  11.99  12.00  11.579935   881664   

    log_ret  
1  0.004626  
2 -0.027292  
3  0.019570  
4 -0.055791  
5 -0.016529  
   Unnamed: 0        Date     Open     High      Low    Close  Adj Close  \
1           1  2017-06-09  21045.0  21271.0  20948.0  21122.0    21122.0   
2           2  2017-06-12  21077.0  21157.0  20910.0  20910.0    20910.0   
3           3  2017-06-13  20989.0  21169.0  20949.0  21089.0    21089.0   
4           4  2017-06-14  21095.0  21198.0  20961.0  20961.0    20961.0   
5           5  2017-06-15  20929.0  20937.0  20732.0  20848.0    20848.0 

  5%|▍         | 2/41 [00:00<00:05,  6.75it/s]


   Unnamed: 0        Date   Open   High    Low  Close  Adj Close   Volume  \
4           4  2017-06-14  12.06  12.24  11.95  12.20  11.772934  7618194   
5           5  2017-06-15  12.20  12.23  11.99  12.00  11.579935   881664   
6           6  2017-06-16  12.14  12.42  12.07  12.42  11.985233  1770075   
7           7  2017-06-19  12.48  12.48  12.21  12.46  12.023831   551385   
8           8  2017-06-20  12.48  12.63  12.32  12.50  12.062431  1170218   

    log_ret  
4 -0.055791  
5 -0.016529  
6  0.034401  
7  0.003215  
8  0.003205  
   Unnamed: 0        Date     Open     High      Low    Close  Adj Close  \
4           4  2017-06-14  21095.0  21198.0  20961.0  20961.0    20961.0   
5           5  2017-06-15  20929.0  20937.0  20732.0  20848.0    20848.0   
6           6  2017-06-16  21000.0  21042.0  20802.0  20941.0    20941.0   
7           7  2017-06-19  20989.0  21115.0  20902.0  21014.0    21014.0   
8           8  2017-06-20  21070.0  21088.0  20810.0  20810.0    20810.0

...

 12%|█▏        | 5/41 [00:00<00:05,  6.69it/s]

    Unnamed: 0        Date   Open   High    Low  Close  Adj Close    Volume  \
6            6  2017-06-16  2.800  2.800  2.672  2.696   2.287398  25150940   
7            7  2017-06-19  2.730  2.782  2.704  2.740   2.324729  15799469   
8            8  2017-06-20  2.750  2.788  2.718  2.726   2.312851  15453934   
9            9  2017-06-21  2.722  2.806  2.664  2.790   2.367151  26619476   
10          10  2017-06-22  2.786  2.786  2.704  2.728   2.314548  20872207   

     log_ret  
6  -0.024186  
7   0.016189  
8  -0.005123  
9   0.023206  
10 -0.022473  
    Unnamed: 0        Date     Open     High      Low    Close  Adj Close  \
6            6  2017-06-16  21000.0  21042.0  20802.0  20941.0    20941.0   
7            7  2017-06-19  20989.0  21115.0  20902.0  21014.0    21014.0   
8            8  2017-06-20  21070.0  21088.0  20810.0  20810.0    20810.0   
9            9  2017-06-21  20769.0  21093.0  20587.0  21072.0    21072.0   
10          10  2017-06-22  20979.0  20979.0  2084

 12%|█▏        | 5/41 [00:00<00:05,  6.52it/s]


   Unnamed: 0        Date       Open       High        Low      Close  \
1           1  2017-06-09  27.500000  28.000000  27.340000  27.969999   
2           2  2017-06-12  27.990000  28.100000  27.790001  27.790001   
3           3  2017-06-13  27.920000  28.059999  27.809999  28.049999   
4           4  2017-06-14  27.990000  28.469999  27.520000  27.520000   
5           5  2017-06-15  27.549999  27.559999  26.510000  26.660000   

   Adj Close  Volume   log_ret  
1  20.589678  512451  0.013317  
2  20.457176  444711 -0.006456  
3  20.648571  351971  0.009312  
4  20.258417  300027 -0.019076  
5  19.625340  553663 -0.031749  
   Unnamed: 0        Date     Open     High      Low    Close  Adj Close  \
1           1  2017-06-09  21045.0  21271.0  20948.0  21122.0    21122.0   
2           2  2017-06-12  21077.0  21157.0  20910.0  20910.0    20910.0   
3           3  2017-06-13  20989.0  21169.0  20949.0  21089.0    21089.0   
4           4  2017-06-14  21095.0  21198.0  20961.0  20961

KeyboardInterrupt: 